# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [9]:
import numpy as np
import pandas as pd
import requests
#import XlsxWriter
import math
import json

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [106]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [14]:
from sec import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN = 'sk_e4b60fff5f6546be8f96d5b265efbe54'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [64]:
symbol = "AAPL"
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


In [65]:
data

{'avgTotalVolume': 64717500,
 'calculationPrice': 'iexlasttrade',
 'change': -0.3,
 'changePercent': -0.0016,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 186.71,
 'iexAskSize': 100,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 186.7,
 'iexCloseTime': 1687550399919,
 'iexLastUpdated': 1687550399919,
 'iexMarketPercent': 0.014874860733379364,
 'iexOpen': 185.53,
 'iexOpenTime': 1687527000995,
 'iexRealtimePrice': 186.7,
 'iexRealtimeSize': 31,
 'iexVolume': 787873,
 'lastTradeTime': 1687550399996,
 'latestPrice': 186.7,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'June 23, 2023',
 'latestUpdate': 1687550399919,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [20]:
price = data['latestPrice']
market_cap = data['marketCap']


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [57]:
my_columns = ['Ticker','Stock Price','Market Capitalization','Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy


In [60]:
#final_dataframe = final_dataframe.append([final_dataframe,],ignore_index=True)

                                          
final_dataframe.loc[1] = [symbol,price,market_cap,'nA']                            

In [61]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
1,AAPL,183.94,2893137445880,nA


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [107]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Symbol'][:5]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    print(data)
    final_dataframe.loc[len(final_dataframe)] = [stock,data['latestPrice'],data['marketCap'],'NA']

{'avgTotalVolume': 4251054, 'calculationPrice': 'close', 'change': 0.29, 'changePercent': 0.00289, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': '3M Co.', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': '15 minute delayed price', 'highTime': 1687550399992, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 100.73, 'iexCloseTime': 1687550394743, 'iexLastUpdated': 1687550394743, 'iexMarketPercent': 0.026923603811679575, 'iexOpen': 102.89, 'iexOpenTime': 1687527007961, 'iexRealtimePrice': 100.73, 'iexRealtimeSize': 300, 'iexVolume': 210293, 'lastTradeTime': 1687550399992, 'latestPrice': 100.72, 'latestSource': 'Close', 'latestTime': 'June 23, 2023', 'latestUpdate': 1687550610137, 'latestVolume': None, 'low': None, 'lowSource': '15 minute delayed price', 'lowTime': 1687546917074, 

In [67]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,119.42,35273780186,NA
1,AAL,16.26,10616154000,NA
2,AAP,65.67,3903677761,NA
3,AAPL,186.70,2936548663400,NA
4,ABBV,135.92,239802253306,NA


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [68]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]

In [105]:
# index = 0 
# index_to_remove = 0
# for i in stocks['Symbol']:
#     index += 1
# stocks['Ticker'] = stocks['Ticker'].drop(index_to_remove)

nan
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
COTY
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
nan
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HFC
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRB
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [104]:
index_to_remove

0

In [109]:
symbol_groups = list( chunks(stocks['Symbol'],100))
symbol_strings = []

for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns= my_columns)
print(symbol_strings)

['MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AAP,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AMD,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ANET,AJG,AIZ,T,ATO,ADSK,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BBWI,BAX,BDX,WRB,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BG,CHRW,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC', 'CNP,CDAY,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DIS,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FITB,FSLR', 'FE,FIS,FI,FLT,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GEHC,GEN,GNRC,GD,GE,GIS,GM,GPC,GILD,GL,GPN,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,

In [110]:


for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb,tsla&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        #print(data[symbol]['quote'])
        final_dataframe.loc[len(final_dataframe)] = [symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'NA']
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,MMM,100.72,55564425696,NA
1,AOS,69.49,10616371371,NA
2,ABT,108.05,187893201632,NA
3,ABBV,135.92,239802253306,NA
4,ACN,298.43,197738427887,NA
...,...,...,...,...
498,YUM,135.60,37979794895,NA
499,ZBRA,264.39,13597667064,NA
500,ZBH,143.99,30031891347,NA
501,ZION,26.58,3936516633,NA


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [111]:
portfolio_size= float(input("Enter the value of the portfolio: "))

Enter the value of the portfolio100000


In [112]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])

In [113]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,MMM,100.72,55564425696,1
1,AOS,69.49,10616371371,2
2,ABT,108.05,187893201632,1
3,ABBV,135.92,239802253306,1
4,ACN,298.43,197738427887,0
...,...,...,...,...
498,YUM,135.60,37979794895,1
499,ZBRA,264.39,13597667064,0
500,ZBH,143.99,30031891347,1
501,ZION,26.58,3936516633,7
